In [ ]:
!pip install flask

In [ ]:
from drl4t_data import download

train_data, test_data = download('nyse.csv')

In [ ]:
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import DQN

model = DQN.load('nyse_dqn_model.pt')

In [ ]:
import pandas as pd
from drl4t_env import DRL4TEnv, Actions

pred = pd.DataFrame(columns=['Symbol', 'Predict'])
for symbol, data in test_data.items():
    env = DRL4TEnv({ symbol: data })
    model.set_env(DummyVecEnv([lambda: env]))

    obs = env.reset()
    done = False

    while(not done):
        action, _ = model.predict(obs)
        obs, _, done, info = env.step(action)

    action, _ = model.predict(obs)
    if action != Actions.Hold.value:
        info['Predict'] = Actions(action).name
        pred = pd.concat([pred, pd.DataFrame(info, index=[info['Date']])])

pred[['Symbol', 'Predict']].to_csv('nyse_dqn_pred.csv')

In [ ]:
from flask import Flask

app = Flask(__name__)

In [ ]:
from flask import render_template
import pandas as pd

@app.route('/', methods=['GET'])
def highlight(): 
    pred = pd.read_csv('nyse_dqn_pred.csv')
    table_html=pred[['Symbol', 'Predict']].to_html()
    return render_template('index.html', table_html=table_html)

In [ ]:
app.run(host='localhost', port=5000)